In [1]:
from prognosis_model import emb_gen, model_lora
from data_inference_hector import Hector_Dataset
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim import Adam

from ct_clip import CTCLIP
from transformer_maskgit import CTViT
from transformers import BertTokenizer, BertModel
from lifelines.utils import concordance_index
from data_inference_hector import Hector_Dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = BertTokenizer.from_pretrained('microsoft/BiomedVLP-CXR-BERT-specialized',do_lower_case=True)
text_encoder = BertModel.from_pretrained("microsoft/BiomedVLP-CXR-BERT-specialized")

text_encoder.resize_token_embeddings(len(tokenizer))
text_encoder.to(device)

image_encoder = CTViT(
    dim = 512,
    codebook_size = 8192,
    image_size = 480,
    patch_size = 20,
    temporal_patch_size = 10,
    spatial_depth = 4,
    temporal_depth = 4,
    dim_head = 32,
    heads = 8
)

image_encoder.to(device)

clip = CTCLIP(
    image_encoder = image_encoder,
    text_encoder = text_encoder,
    dim_image = 294912,
    dim_text = 768,
    dim_latent = 512,
    extra_latent_projection = False,         # whether to use separate projections for text-to-image vs image-to-text comparisons (CLOOB)
    use_mlm=False,
    downsample_image_embeds = False,
    use_all_token_embeds = False,
)

clip.load("/share/sda/mohammadqazi/project/CTscan_prognosis_VLM-main/docs/CT-CLIP_v2.pt")
clip.to(device)

# model = emb_gen(clip, device = device)

/home/mohammadqazi/.conda/envs/ct_rate/lib/python3.10/site-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:261: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/home/mohammadqazi/.conda/envs/ct_rate/lib/python3.10/site-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:391: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/home/mohammadqazi/.conda/envs/ct_rate/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/mohammadqazi/.conda/envs/ct_rate/lib/python3.10/site-packages/transformers/modeling_utils.py:463: FutureWarning: You are using `tor

CTCLIP(
  (text_transformer): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.25, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.25, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw

In [4]:
hect_dataset = Hector_Dataset(data_folder = "/share/sda/mohammadqazi/project/hector/pre_processed/",  
                csv_file ="/share/sda/mohammadqazi/project/CTscan_prognosis_VLM-main/docs/TNM_hector_prompts.csv")

loader = DataLoader(hect_dataset, batch_size=8, shuffle=False)

In [8]:
from torchinfo import summary

img_emb, text, relapse, RFS, _ = next(iter(loader))
img_emb = img_emb.to(device)
text_tokens = tokenizer(
    text, 
    return_tensors="pt", 
    padding="max_length", 
    truncation=True, 
    max_length=512
).to(device)
relapse = relapse.to(device)
RFS = RFS.to(device)
summary(model, input_data=[text_tokens,img_emb ], depth=8, col_names=["input_size", "output_size", "num_params"],)

Layer (type:depth-idx)                                       Input Shape               Output Shape              Param #
emb_gen                                                      --                        [8, 512]                  --
├─CTCLIP: 1-1                                                --                        [8, 24, 24, 512]          302,776,321
│    └─BertModel: 2-1                                        [8, 512]                  [8, 768]                  --
│    │    └─BertEmbeddings: 3-1                              --                        [8, 512, 768]             --
│    │    │    └─Embedding: 4-1                              [8, 512]                  [8, 512, 768]             23,440,896
│    │    │    └─Embedding: 4-2                              [8, 512]                  [8, 512, 768]             1,536
│    │    │    └─Embedding: 4-3                              [1, 512]                  [1, 512, 768]             393,216
│    │    │    └─LayerNorm: 4-4           

In [4]:
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType


peft_config = LoraConfig(
    inference_mode=False, r=8, lora_alpha=64, lora_dropout=0.2, target_modules=["to_q", "to_kv"]
)
model_peft = get_peft_model(model, peft_config)

In [10]:
for name, param in model_peft.named_parameters():
    if param.requires_grad:
        print(name)

base_model.model.clip.visual_transformer.enc_spatial_transformer.layers.0.1.to_q.lora_A.default.weight
base_model.model.clip.visual_transformer.enc_spatial_transformer.layers.0.1.to_q.lora_B.default.weight
base_model.model.clip.visual_transformer.enc_spatial_transformer.layers.0.1.to_kv.lora_A.default.weight
base_model.model.clip.visual_transformer.enc_spatial_transformer.layers.0.1.to_kv.lora_B.default.weight
base_model.model.clip.visual_transformer.enc_spatial_transformer.layers.1.1.to_q.lora_A.default.weight
base_model.model.clip.visual_transformer.enc_spatial_transformer.layers.1.1.to_q.lora_B.default.weight
base_model.model.clip.visual_transformer.enc_spatial_transformer.layers.1.1.to_kv.lora_A.default.weight
base_model.model.clip.visual_transformer.enc_spatial_transformer.layers.1.1.to_kv.lora_B.default.weight
base_model.model.clip.visual_transformer.enc_spatial_transformer.layers.2.1.to_q.lora_A.default.weight
base_model.model.clip.visual_transformer.enc_spatial_transformer.laye

In [7]:
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

num_time_bins = 12
peft_config = LoraConfig(
    inference_mode=False, r=8, lora_alpha=64, lora_dropout=0.2, target_modules=["to_q", "to_kv"]
)

model = model_lora(clip, device, peft_config, num_time_bins)
model.to(device)

model_lora(
  (clip): PeftModel(
    (base_model): LoraModel(
      (model): CTCLIP(
        (text_transformer): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.25, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0-11): 12 x BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=True)
                    (key): Linear(in_features=768, out_features=768, bias=True)
                    (value): Linear(in_features=768, out_features=768, bias=True)
                    (dropout): Dropout(p=0.25, inplace=False)
             

In [8]:
from torchinfo import summary

hect_dataset = Hector_Dataset(data_folder = "/share/sda/mohammadqazi/project/hector/pre_processed/",  
                csv_file ="/share/sda/mohammadqazi/project/CTscan_prognosis_VLM-main/docs/TNM_hector_prompts.csv")

loader = DataLoader(hect_dataset, batch_size=8, shuffle=False)

img_emb, text, relapse, RFS, _ = next(iter(loader))
img_emb = img_emb.to(device)
text_tokens = tokenizer(
    text, 
    return_tensors="pt", 
    padding="max_length", 
    truncation=True, 
    max_length=512
).to(device)
relapse = relapse.to(device)
RFS = RFS.to(device)

summary(model, input_data=[text_tokens,img_emb ], depth=8, col_names=["input_size", "output_size", "num_params"],)

Layer (type:depth-idx)                                                 Input Shape               Output Shape              Param #
model_lora                                                             --                        [8, 13]                   --
├─PeftModel: 1-1                                                       --                        [8, 24, 24, 512]          --
│    └─LoraModel: 2-1                                                  --                        --                        --
│    │    └─CTCLIP: 3-1                                                --                        [8, 24, 24, 512]          302,776,321
│    │    │    └─BertModel: 4-1                                        [8, 512]                  [8, 768]                  --
│    │    │    │    └─BertEmbeddings: 5-1                              --                        [8, 512, 768]             --
│    │    │    │    │    └─Embedding: 6-1                              [8, 512]                  [8, 512

In [10]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

clip.base_model.model.visual_transformer.enc_spatial_transformer.layers.0.1.to_q.lora_A.default.weight
clip.base_model.model.visual_transformer.enc_spatial_transformer.layers.0.1.to_q.lora_B.default.weight
clip.base_model.model.visual_transformer.enc_spatial_transformer.layers.0.1.to_kv.lora_A.default.weight
clip.base_model.model.visual_transformer.enc_spatial_transformer.layers.0.1.to_kv.lora_B.default.weight
clip.base_model.model.visual_transformer.enc_spatial_transformer.layers.1.1.to_q.lora_A.default.weight
clip.base_model.model.visual_transformer.enc_spatial_transformer.layers.1.1.to_q.lora_B.default.weight
clip.base_model.model.visual_transformer.enc_spatial_transformer.layers.1.1.to_kv.lora_A.default.weight
clip.base_model.model.visual_transformer.enc_spatial_transformer.layers.1.1.to_kv.lora_B.default.weight
clip.base_model.model.visual_transformer.enc_spatial_transformer.layers.2.1.to_q.lora_A.default.weight
clip.base_model.model.visual_transformer.enc_spatial_transformer.laye

In [17]:
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np
import pandas as pd
from functools import partial
import os

class Hector_Dataset_Subset(Dataset):
    def __init__(self, samples):
        self.samples = samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        return self.samples[index]

class Hector_Dataset_emb(Dataset):
    def __init__(self, emd_path, csv_file):
        self.emd = np.load(emd_path , allow_pickle=True).item()
        self.dataframe = pd.read_csv(csv_file)
        self.paths=[]
        self.samples = self.prepare_samples()
        self.nii_to_tensor = partial(self.to_tensor)

    def prepare_samples(self):
        samples = []

        for index, row in self.dataframe.iterrows():
            filename = row['PatientID'] + "_ct_roi.npz"
            # filepath = os.path.join(self.data_folder, filename)
            image_embedding = self.emd[filename]['image_embedding']
            text_embedding = self.emd[filename]['text_embedding']
            fold = row['fold']
            samples.append((image_embedding, text_embedding, row['Relapse'], row['RFS'], filename, fold))
        return samples

    def __len__(self):
        return len(self.samples)
    
    def train_val_split(self, fold):
        train_samples = []
        val_samples = []
        for sample in self.samples:
            if sample[5] == fold:
                val_samples.append(sample)
            else:
                train_samples.append(sample)
        return Hector_Dataset_Subset(train_samples), Hector_Dataset_Subset(val_samples)

    def to_tensor(self, emb):
        # img_data = np.load(path)['arr_0']
        tensor = torch.tensor(emb)
        # tensor = tensor.unsqueeze(0)
        return tensor

    def __getitem__(self, index):
        image_embedding, text_embedding, relapse, RFS, filename, fold= self.samples[index]
        text_embedding_tensor = self.to_tensor(text_embedding)
        image_embedding_tensor = self.to_tensor(image_embedding)
        return image_embedding_tensor, text_embedding_tensor, relapse, RFS, filename, fold

hect_dataset = Hector_Dataset_emb(emd_path = '/share/sda/mohammadqazi/project/CTscan_prognosis_VLM-main/docs/embeddings/embeddings.npy',  
                csv_file ="/share/sda/mohammadqazi/project/CTscan_prognosis_VLM-main/docs/TNM_hector_prompts.csv")

In [18]:
train_dataset, test_dataset = hect_dataset.train_val_split(fold=0)
                            train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [19]:
for img_emb, text_emb, relapse, RFS, filename, fold in train_loader:
    print(img_emb.shape)
    print(text_emb.shape)
    print(relapse)
    print(RFS)
    print(filename)
    print(fold)
    break

torch.Size([64, 512])
torch.Size([64, 768])
tensor([0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([ 903,  591, 1670, 1429, 1354, 1296,  175, 1087,  984, 3534, 5484,  419,
        1005, 3606, 1152, 1174, 1307, 3069, 3148, 1000,  195, 2615, 1357,   88,
        3533, 3022, 1297, 2856, 1882,  377, 1410, 1299,  888,  996, 2566, 4425,
        2858,  762, 1833,  378, 1201, 1459,  929, 2847,  493, 2201,   97,  325,
        1080, 2806, 2518, 3861, 5137,  635, 1557,   77, 5116, 2843, 1215, 2562,
        1421, 2421, 1081, 1847])
('CHUM-030_ct_roi.npz', 'MDA-096_ct_roi.npz', 'MDA-048_ct_roi.npz', 'CHUP-001_ct_roi.npz', 'HMR-040_ct_roi.npz', 'MDA-108_ct_roi.npz', 'CHUP-013_ct_roi.npz', 'CHUM-035_ct_roi.npz', 'MDA-113_ct_roi.npz', 'MDA-078_ct_roi.npz', 'MDA-168_ct_roi.npz', 'CHUS-049_ct_roi.npz', 'CHUS-055_ct_roi.npz', 'MDA-064_ct_r

## Parameter Comparsion

In [14]:

import torch
from torch import nn

model_1 = nn.Sequential(
    nn.Linear(512 + 512, 512),
    nn.GELU(),
    nn.Linear(512, 128),
)

model_2 = nn.Sequential(
    nn.Conv1d(
        in_channels=(512 + 512),
        out_channels=512,
        kernel_size=3,
        padding=1,
    ),
    nn.GELU(),
    nn.Conv1d(
        in_channels=512,
        out_channels=128,
        kernel_size=3,
        padding=1,
    ),
)

from torchinfo import summary

summary(model_1, input_data=torch.rand(1, 512*2), depth=8, col_names=["input_size", "output_size", "num_params"],)

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
Sequential                               [1, 1024]                 [1, 128]                  --
├─Linear: 1-1                            [1, 1024]                 [1, 512]                  524,800
├─GELU: 1-2                              [1, 512]                  [1, 512]                  --
├─Linear: 1-3                            [1, 512]                  [1, 128]                  65,664
Total params: 590,464
Trainable params: 590,464
Non-trainable params: 0
Total mult-adds (M): 0.59
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 2.36
Estimated Total Size (MB): 2.37

In [15]:
summary(model_2, input_data=torch.rand(1, 512+512, 1), depth=8, col_names=["input_size", "output_size", "num_params"],)

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
Sequential                               [1, 1024, 1]              [1, 128, 1]               --
├─Conv1d: 1-1                            [1, 1024, 1]              [1, 512, 1]               1,573,376
├─GELU: 1-2                              [1, 512, 1]               [1, 512, 1]               --
├─Conv1d: 1-3                            [1, 512, 1]               [1, 128, 1]               196,736
Total params: 1,770,112
Trainable params: 1,770,112
Non-trainable params: 0
Total mult-adds (M): 1.77
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 7.08
Estimated Total Size (MB): 7.09

# LORA

In [1]:
import pandas as pd
import numpy as np
import argparse
from tqdm import tqdm, trange
import os

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim import Adam, AdamW
from torchinfo import summary

from utils import make_time_bins
from utils import encode_survival, mtlr_neg_log_likelihood, make_optimizer
from utils import mtlr_survival, mtlr_risk, roc_auc_at_times, brier_score_at_times
from prognosis_model import model_lora, model_lora_again
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR


from ct_clip import CTCLIP
from transformer_maskgit import CTViT
from transformers import BertTokenizer, BertModel
from lifelines.utils import concordance_index
from data_inference_hector import Hector_Dataset_lora

from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

seed = 42
torch.manual_seed(seed) 
generator = torch.Generator().manual_seed(seed)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

tokenizer = BertTokenizer.from_pretrained('microsoft/BiomedVLP-CXR-BERT-specialized',do_lower_case=True)
text_encoder = BertModel.from_pretrained("microsoft/BiomedVLP-CXR-BERT-specialized")

text_encoder.resize_token_embeddings(len(tokenizer))
text_encoder.to(device)

image_encoder = CTViT(
    dim = 512,
    codebook_size = 8192,
    image_size = 480,
    patch_size = 20,
    temporal_patch_size = 10,
    spatial_depth = 4,
    temporal_depth = 4,
    dim_head = 32,
    heads = 8
)

image_encoder.to(device)

clip = CTCLIP(
    image_encoder = image_encoder,
    text_encoder = text_encoder,
    dim_image = 294912,
    dim_text = 768,
    dim_latent = 512,
    extra_latent_projection = False,         # whether to use separate projections for text-to-image vs image-to-text comparisons (CLOOB)
    use_mlm=False,
    downsample_image_embeds = False,
    use_all_token_embeds = False,
)

clip.load("/share/sda/mohammadqazi/project/CTscan_prognosis_VLM-main/docs/CT-CLIP_v2.pt")
clip.to(device)

/home/mohammadqazi/.conda/envs/ct_rate/lib/python3.10/site-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:261: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/home/mohammadqazi/.conda/envs/ct_rate/lib/python3.10/site-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:391: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/home/mohammadqazi/.conda/envs/ct_rate/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/mohammadqazi/.conda/envs/ct_rate/lib/python3.10/site-packages/transformers/modeling_utils.py:463: FutureWarning: You are using `tor

CTCLIP(
  (text_transformer): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.25, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.25, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw

In [2]:
output_file = "baseline.txt"

with open(output_file, "w") as f:
    for name, module in clip.named_modules():
        f.write(name + "\n")

In [3]:
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

peft_config = LoraConfig(
    inference_mode=False, r=8, lora_alpha=64, lora_dropout=0.2, target_modules=["to_q", "to_kv"]
)

clip = get_peft_model(clip, peft_config)

In [4]:
output_file = "lora.txt"

with open(output_file, "w") as f:
    for name, module in clip.named_modules():
        f.write(name + "\n")

In [9]:
clip

PeftModel(
  (base_model): LoraModel(
    (model): CTCLIP(
      (text_transformer): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.25, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.25, inplace=False)
                )
                (output): BertSelfOutput(
             

In [4]:
for name, module in clip.named_modules():
    print(name)


base_model
base_model.model
base_model.model.text_transformer
base_model.model.text_transformer.embeddings
base_model.model.text_transformer.embeddings.word_embeddings
base_model.model.text_transformer.embeddings.position_embeddings
base_model.model.text_transformer.embeddings.token_type_embeddings
base_model.model.text_transformer.embeddings.LayerNorm
base_model.model.text_transformer.embeddings.dropout
base_model.model.text_transformer.encoder
base_model.model.text_transformer.encoder.layer
base_model.model.text_transformer.encoder.layer.0
base_model.model.text_transformer.encoder.layer.0.attention
base_model.model.text_transformer.encoder.layer.0.attention.self
base_model.model.text_transformer.encoder.layer.0.attention.self.query
base_model.model.text_transformer.encoder.layer.0.attention.self.key
base_model.model.text_transformer.encoder.layer.0.attention.self.value
base_model.model.text_transformer.encoder.layer.0.attention.self.dropout
base_model.model.text_transformer.encoder.

In [2]:
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
target =  ["enc_spatial_transformer.layers.0.1.to_q", "enc_spatial_transformer.layers.0.1.to_kv", "enc_spatial_transformer.layers.0.1.to_out",
"enc_spatial_transformer.layers.1.1.to_q", "enc_spatial_transformer.layers.1.1.to_kv", "enc_spatial_transformer.layers.1.1.to_out",
"enc_spatial_transformer.layers.2.1.to_q", "enc_spatial_transformer.layers.2.1.to_kv", "enc_spatial_transformer.layers.2.1.to_out",
"enc_spatial_transformer.layers.3.1.to_q", "enc_spatial_transformer.layers.3.1.to_kv", "enc_spatial_transformer.layers.3.1.to_out",]
peft_config = LoraConfig(
    inference_mode=False, r=8, lora_alpha=64, lora_dropout=0.2, target_modules=target
)

clip = get_peft_model(clip, peft_config)

output_file = "new_lora.txt"

with open(output_file, "w") as f:
    for name, module in clip.named_modules():
        f.write(name + "\n")

In [1]:
clip

NameError: name 'clip' is not defined

# Parameter Check

In [15]:
import pandas as pd
import numpy as np
import argparse
from tqdm import tqdm, trange
import os

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim import Adam, AdamW
from torchinfo import summary

from utils import make_time_bins
from utils import encode_survival, mtlr_neg_log_likelihood, make_optimizer
from utils import mtlr_survival, mtlr_risk, roc_auc_at_times, brier_score_at_times
from prognosis_model import model_lora, model_lora_again
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR


from ct_clip import CTCLIP
from transformer_maskgit import CTViT
from transformers import BertTokenizer, BertModel
from lifelines.utils import concordance_index
from data_inference_hector import Hector_Dataset_lora

from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

num_time_bins = 12
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

tokenizer = BertTokenizer.from_pretrained('microsoft/BiomedVLP-CXR-BERT-specialized',do_lower_case=True)
text_encoder = BertModel.from_pretrained("microsoft/BiomedVLP-CXR-BERT-specialized")

text_encoder.resize_token_embeddings(len(tokenizer))
text_encoder.to(device)

image_encoder = CTViT(
    dim = 512,
    codebook_size = 8192,
    image_size = 480,
    patch_size = 20,
    temporal_patch_size = 10,
    spatial_depth = 4,
    temporal_depth = 4,
    dim_head = 32,
    heads = 8
)

image_encoder.to(device)

clip = CTCLIP(
    image_encoder = image_encoder,
    text_encoder = text_encoder,
    dim_image = 294912,
    dim_text = 768,
    dim_latent = 512,
    extra_latent_projection = False,         # whether to use separate projections for text-to-image vs image-to-text comparisons (CLOOB)
    use_mlm=False,
    downsample_image_embeds = False,
    use_all_token_embeds = False,
)

clip.load("/share/sda/mohammadqazi/project/CTscan_prognosis_VLM-main/docs/CT-CLIP_v2.pt")
clip.to(device)

/home/mohammadqazi/.conda/envs/ct_rate/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/mohammadqazi/.conda/envs/ct_rate/lib/python3.10/site-packages/transformers/modeling_utils.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are

CTCLIP(
  (text_transformer): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.25, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.25, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw

In [18]:
# peft_config = LoraConfig(
#     inference_mode=False, r=8, lora_alpha=64, lora_dropout=0.2, target_modules=["to_q", "to_kv"]
# )

# target =  ["enc_spatial_transformer.layers.0.1.to_q", "enc_spatial_transformer.layers.0.1.to_kv", "enc_spatial_transformer.layers.0.1.to_out",
#     "enc_spatial_transformer.layers.1.1.to_q", "enc_spatial_transformer.layers.1.1.to_kv", "enc_spatial_transformer.layers.1.1.to_out",
#     "enc_spatial_transformer.layers.2.1.to_q", "enc_spatial_transformer.layers.2.1.to_kv", "enc_spatial_transformer.layers.2.1.to_out",
#     "enc_spatial_transformer.layers.3.1.to_q", "enc_spatial_transformer.layers.3.1.to_kv", "enc_spatial_transformer.layers.3.1.to_out",]

target = ["to_q", "to_kv"]    
peft_config = LoraConfig(
    inference_mode=False, r=2, lora_alpha=32, lora_dropout=0.2, target_modules=target, 
    # task_type='QUESTION_ANS'
)

model_ = model_lora_again(clip, device, peft_config, num_time_bins)
model_.to(device)

# from peft import get_peft_model

# model = get_peft_model(model_, peft_config)
# model.to(device)

model_lora_again(
  (clip): CTCLIP(
    (text_transformer): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.25, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.25, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=Tr

In [4]:
from data_inference_hector import Hector_Dataset_lora

df = pd.read_csv("/share/sda/mohammadqazi/project/CTscan_prognosis_VLM-main/docs/TNM_hector_prompts.csv")

hect_dataset = Hector_Dataset_lora(data_folder = "/share/sda/mohammadqazi/project/hector/pre_processed/",  
            csv_file ="/share/sda/mohammadqazi/project/CTscan_prognosis_VLM-main/docs/TNM_hector_prompts.csv")

train_dataset, test_dataset = hect_dataset.train_val_split(fold=0)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

time_bins = make_time_bins(df['RFS'].values, event=df['Relapse'].values, num_bins=num_time_bins)

In [19]:
img_emb, text_emb, relapse, RFS, _, _ = next(iter(train_loader))
img_emb = img_emb.to(device)
text_tokens=tokenizer(text_emb, return_tensors="pt", padding="max_length", truncation=True, max_length=512).to(device)
y = encode_survival(RFS, relapse, time_bins).to(device)
relapse = relapse.to(device)
RFS = RFS.to(device)
summary(model_, input_data=[img_emb, text_tokens ], depth=8, col_names=["input_size", "output_size", "num_params"],)

Layer (type:depth-idx)                                       Input Shape               Output Shape              Param #
model_lora_again                                             [4, 1, 240, 480, 480]     [4, 13]                   --
├─CTCLIP: 1-1                                                --                        [4, 24, 24, 512]          302,776,321
│    └─BertModel: 2-1                                        [4, 512]                  [4, 768]                  --
│    │    └─BertEmbeddings: 3-1                              --                        [4, 512, 768]             --
│    │    │    └─Embedding: 4-1                              [4, 512]                  [4, 512, 768]             23,440,896
│    │    │    └─Embedding: 4-2                              [4, 512]                  [4, 512, 768]             1,536
│    │    │    └─Embedding: 4-3                              [1, 512]                  [1, 512, 768]             393,216
│    │    │    └─LayerNorm: 4-4           

In [13]:
# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total Parameters: {total_params:,}")
print(f"Trainable Parameters (LoRA applied): {trainable_params:,}")

print(model.print_trainable_parameters())

Total Parameters: 439,953,189
Trainable Parameters (LoRA applied): 28,672
trainable params: 28,672 || all params: 439,953,189 || trainable%: 0.006517056977168542
None


# Segmentation

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

###############################################################################
# A helper block that upsamples a feature map and fuses it with a skip connection.
# Here we simply upsample with a ConvTranspose3d, concatenate the skip feature,
# then apply a convolution + BN + ReLU.
###############################################################################
class DecoderBlock3D(nn.Module):
    def __init__(self, in_channels, skip_channels, out_channels, 
                 up_kernel_size=2, conv_kernel_size=3):
        super().__init__()
        self.upsample = nn.ConvTranspose3d(in_channels, out_channels, 
                                           kernel_size=up_kernel_size, stride=up_kernel_size)
        # We use concatenation so the channel dimension is (upsampled + skip)
        self.conv = nn.Sequential(
            nn.Conv3d(out_channels + skip_channels, out_channels, 
                      kernel_size=conv_kernel_size, padding=conv_kernel_size // 2),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x, skip):
        x = self.upsample(x)
        # Make sure that x and skip have the same spatial dimensions;
        # if not, you might need to crop or interpolate.
        if skip is not None:
            # Concatenate along the channel dimension
            x = torch.cat([x, skip], dim=1)
        x = self.conv(x)
        return x

###############################################################################
# A simple output block that maps the final features to segmentation classes.
###############################################################################
class UnetOutBlock3D(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Conv3d(in_channels, out_channels, kernel_size=1)
    
    def forward(self, x):
        return self.conv(x)

###############################################################################
# The main network that mimics a UNETR‐style design.
#
# - The main feature comes from the visual transformer as "enc_image" (which is
#   a 5D tensor: [B, D, H, W, C]). We first permute it to [B, C, D, H, W].
#
# - Three skip connections are taken from transformer hidden states.
#   (We assume that each hidden state is originally of shape [B, num_tokens+1, C],
#    with the first token being a [CLS] token that we remove.)
#
# - The helper method `proj_feat` reshapes a hidden state token sequence of shape
#   [B, num_tokens, C] into a spatial map of shape [B, C, D, H, W], where D×H×W
#   is provided via the attribute `feat_size`.
###############################################################################
class MyUNETR3D(nn.Module):
    def __init__(self, hidden_size=512, 
                 feat_size=(24, 24, 24),  # spatial grid dimensions for the hidden tokens
                 base_channels=64,      # base number of channels in the decoder
                 out_channels=2         # number of segmentation classes
                ):
        super().__init__()
        self.hidden_size = hidden_size
        self.feat_size = feat_size  # for example, if the transformer produces 24*24*24 tokens
        # In our design we assume that both the main branch and each skip connection
        # have "hidden_size" channels.
        #
        # We'll define three decoder blocks:
        #  - First decoder block: upsamples from the main feature (enc_image) and fuses with the deepest skip.
        #  - Second decoder block: fuses with a mid-level skip.
        #  - Third decoder block: fuses with the shallow skip.
        #
        # Then an extra upsampling + conv brings us to the desired segmentation resolution.
        self.dec_block1 = DecoderBlock3D(in_channels=hidden_size, 
                                         skip_channels=hidden_size, 
                                         out_channels=base_channels * 4)
        self.dec_block2 = DecoderBlock3D(in_channels=base_channels * 4, 
                                         skip_channels=hidden_size, 
                                         out_channels=base_channels * 2)
        self.dec_block3 = DecoderBlock3D(in_channels=base_channels * 2, 
                                         skip_channels=hidden_size, 
                                         out_channels=base_channels)
        # One more upsampling (without skip connection) to reach a higher spatial resolution:
        self.up_conv = nn.ConvTranspose3d(base_channels, base_channels, kernel_size=2, stride=2)
        self.out_block = UnetOutBlock3D(base_channels, out_channels)
    
    def proj_feat(self, x):
        """
        Projects a transformer hidden state (after removing any [CLS] token) into
        a 3D spatial feature map.
        
        Args:
            x: Tensor of shape [B, num_tokens, hidden_size]. Here num_tokens must equal
               feat_size[0] * feat_size[1] * feat_size[2].
        
        Returns:
            A tensor of shape [B, hidden_size, D, H, W].
        """
        B, num_tokens, C = x.shape
        D, H, W = self.feat_size
        assert num_tokens == D * H * W, f"Expected {D*H*W} tokens, but got {num_tokens}."
        x = x.view(B, D, H, W, C)      # [B, D, H, W, C]
        x = x.permute(0, 4, 1, 2, 3).contiguous()  # [B, C, D, H, W]
        return x
    
    def forward(self, enc_image, hidden_states):
        """
        Args:
            enc_image: The spatial feature from the visual transformer.
                       Expected shape: [B, D, H, W, C]  (e.g., [4, 24, 24, 24, 512])
            hidden_states: A list of transformer hidden states to be used as skip connections.
                           Each element is expected to be of shape [B, num_tokens+1, C]
                           (the first token is assumed to be a classification token that will be dropped).
                           For example, use hidden_states[9], hidden_states[6], hidden_states[3].
        
        Returns:
            seg_out: The segmentation logits.
        """
        # --- Main branch ---
        # Permute enc_image to get channels-first:
        x_main = enc_image.permute(0, 4, 1, 2, 3).contiguous()  # [B, C, D, H, W]
        
        # --- Prepare skip connections ---
        # Remove the first token (assumed to be a [CLS] token) and project the rest into a 3D feature map.
        # Here we assume that:
        #   - hidden_states[9] is used as the deepest skip connection,
        #   - hidden_states[6] as the intermediate,
        #   - hidden_states[3] as the shallow one.
        skip_deep = self.proj_feat(hidden_states[9][:, 1:, :])  # shape: [B, hidden_size, D, H, W]
        skip_mid  = self.proj_feat(hidden_states[6][:, 1:, :])
        skip_shallow = self.proj_feat(hidden_states[3][:, 1:, :])
        
        # --- Decoder: fuse the main feature with the skip connections ---
        x = self.dec_block1(x_main, skip_deep)
        x = self.dec_block2(x, skip_mid)
        x = self.dec_block3(x, skip_shallow)
        
        # Optionally, upsample once more to get a higher resolution segmentation map.
        x = self.up_conv(x)
        seg_out = self.out_block(x)
        return seg_out

###############################################################################
# Example usage:
#
# Suppose that your visual transformer returns:
#  - enc_image: a tensor of shape [B, 24, 24, 24, 512]
#  - hidden_states: a list (with at least indices 3, 6, 9) where each element is of shape
#    [B, num_tokens+1, 512]. For example, if the tokens form a 3D grid of 24x24x24,
#    then num_tokens == 24*24*24.
###############################################################################
if __name__ == '__main__':
    # Create dummy inputs:
    B = 4
    D = H = W = 24
    C = 512
    num_tokens = D * H * W  # 24*24*24 = 13824 tokens
    
    # Dummy enc_image: already spatial, shape [B, D, H, W, C]
    enc_image = torch.randn(B, D, H, W, C)
    
    # Create dummy hidden states.
    # For each skip connection we simulate a tensor with an extra [CLS] token.
    # For simplicity, we use the same spatial resolution here.
    dummy_hidden_state = torch.randn(B, 576, C)
    # Build a list for indices 3, 6, 9 (the values don’t matter here)
    hidden_states = {3: dummy_hidden_state, 6: dummy_hidden_state, 9: dummy_hidden_state}
    # For convenience, we put them in a list where index corresponds to the transformer layer index.
    # In practice your transformer might return a list; here we just construct one.
    hs_list = [None] * 10
    hs_list[3] = hidden_states[3]
    hs_list[6] = hidden_states[6]
    hs_list[9] = hidden_states[9]
    
    # Initialize the network
    net = MyUNETR3D(hidden_size=C, feat_size=(D, H, W), base_channels=64, out_channels=2)
    
    # Forward pass:
    seg_logits = net(enc_image, hs_list)
    print("Segmentation output shape:", seg_logits.shape)
    # For example, you might expect an output of shape [B, out_channels, (upsampled D), (upsampled H), (upsampled W)]


AssertionError: Expected 13824 tokens, but got 575.

In [1]:
from monai.networks.blocks.unetr_block import UnetrBasicBlock, UnetrPrUpBlock, UnetrUpBlock
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

encoder1 = UnetrBasicBlock(
    spatial_dims=3,
    in_channels=1,
    out_channels=16,
    kernel_size=3,
    stride=1,
    norm_name='instance',
    res_block=True,
)
encoder1.to(device)

UnetrBasicBlock(
  (layer): UnetResBlock(
    (conv1): Convolution(
      (conv): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    )
    (conv2): Convolution(
      (conv): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    )
    (lrelu): LeakyReLU(negative_slope=0.01, inplace=True)
    (norm1): InstanceNorm3d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (norm2): InstanceNorm3d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (conv3): Convolution(
      (conv): Conv3d(1, 16, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
    )
    (norm3): InstanceNorm3d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  )
)

In [1]:
import torch
from monai.networks.blocks.unetr_block import UnetrBasicBlock
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

encoder1 = UnetrBasicBlock(
    spatial_dims=3,
    in_channels=1,
    out_channels=16,
    kernel_size=3,
    stride=1,
    norm_name="instance",
    res_block=True,
)
encoder1.to(device)

# Create a dummy input with shape (batch_size, 1, 240, 480, 480)
dummy_input = torch.randn(1, 1, 240, 480, 480).to(device)  # batch_size = 2

# Forward the dummy input through the block.
output = encoder1(dummy_input)

print("Input shape: ", dummy_input.shape)
print("Output shape:", output.shape)


Input shape:  torch.Size([1, 1, 240, 480, 480])
Output shape: torch.Size([1, 16, 240, 480, 480])


In [2]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def proj_feat(x, hidden_size, feat_size):
    new_view = (x.size(0), *feat_size, hidden_size)
    x = x.view(new_view)
    new_axes = (0, len(x.shape) - 1) + tuple(d + 1 for d in range(len(feat_size)))
    x = x.permute(new_axes).contiguous()
    return x

dummy_input = torch.randn(4, 13824, 512).to(device)  # batch_size = 2
proj_feat(dummy_input[:,:,:], 512, (24, 24, 24)).shape

torch.Size([4, 512, 24, 24, 24])

In [3]:
import torch
from monai.networks.blocks.unetr_block import UnetrPrUpBlock, UnetrUpBlock

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

encoder2 = UnetrPrUpBlock(
    spatial_dims=3,
    in_channels=512,
    out_channels=16 * 2,
    num_layer=2,
    kernel_size=3,
    stride=1,
    upsample_kernel_size=2,
    norm_name='instance',
    conv_block=True,
    res_block=True,
)
encoder2.to(device)

# Create a dummy input with shape (batch_size, 1, 240, 480, 480)
dummy_input = torch.randn(4, 512, 24, 24, 24).to(device)  # batch_size = 2
print("Input shape: ", dummy_input.shape)

# Forward the dummy input through the block.
output = encoder2(dummy_input)
print("Output shape:", output.shape)

Input shape:  torch.Size([4, 512, 24, 24, 24])
Output shape: torch.Size([4, 32, 192, 192, 192])


In [1]:
import torch
from monai.networks.blocks.unetr_block import UnetrPrUpBlock, UnetrUpBlock

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

encoder3 = UnetrPrUpBlock(
    spatial_dims=3,
    in_channels=512,
    out_channels=16 * 4,
    num_layer=2,
    kernel_size=3,
    stride=1,
    upsample_kernel_size=2,
    norm_name='instance',
    conv_block=True,
    res_block=True,
)
encoder3.to(device)

# Create a dummy input with shape (batch_size, 1, 240, 480, 480)
dummy_input = torch.randn(1, 512, 24, 24, 24).to(device)  # batch_size = 2
print("Input shape: ", dummy_input.shape)

# Forward the dummy input through the block.
output = encoder3(dummy_input)
print("Output shape:", output.shape)

Input shape:  torch.Size([1, 512, 24, 24, 24])
Output shape: torch.Size([1, 64, 192, 192, 192])


In [1]:
import torch
from monai.networks.blocks.unetr_block import UnetrPrUpBlock, UnetrUpBlock

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

encoder4 = UnetrPrUpBlock(
    spatial_dims=3,
    in_channels=512,
    out_channels=16 * 8,
    num_layer=2,
    kernel_size=3,
    stride=1,
    upsample_kernel_size=2,
    norm_name='instance',
    conv_block=True,
    res_block=True,
)
encoder4.to(device)

# Create a dummy input with shape (batch_size, 1, 240, 480, 480)
dummy_input = torch.randn(1, 512, 24, 24, 24).to(device)  # batch_size = 2
print("Input shape: ", dummy_input.shape)

# Forward the dummy input through the block.
output = encoder4(dummy_input)
print("Output shape:", output.shape)

Input shape:  torch.Size([1, 512, 24, 24, 24])
Output shape: torch.Size([1, 128, 192, 192, 192])


In [7]:
x.shape

torch.Size([96, 576, 512])

In [1]:
from ct_clip.ct_clip_seg import seg_model
model = seg_model()

In [2]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

seg_model(
  (encoder1): UnetrBasicBlock(
    (layer): UnetResBlock(
      (conv1): Convolution(
        (conv): Conv3d(512, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      )
      (conv2): Convolution(
        (conv): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      )
      (lrelu): LeakyReLU(negative_slope=0.01, inplace=True)
      (norm1): InstanceNorm3d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (norm2): InstanceNorm3d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (conv3): Convolution(
        (conv): Conv3d(512, 16, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      )
      (norm3): InstanceNorm3d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    )
  )
  (encoder2): UnetrPrUpBlock(
    (transp_conv_init): Convolution(
      (conv): ConvTranspose3d(512, 32, kernel_size=(2, 2, 2), stride=(2, 2, 2), bias=False)
  

In [3]:
model(torch.randn(1, 1, 240, 480, 480).to(device), [torch.randn(1, 13824, 512).to(device), torch.randn(1, 13824, 512).to(device), torch.randn(1, 13824, 512).to(device), torch.randn(1, 13824, 512).to(device)])

RuntimeError: Given groups=1, weight of size [16, 512, 3, 3, 3], expected input[1, 1, 240, 480, 480] to have 512 channels, but got 1 channels instead

# Parameter CHeck

In [2]:
import pandas as pd
import numpy as np
import argparse
from tqdm import tqdm, trange
import os

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim import Adam, AdamW
from torchinfo import summary

from utils import make_time_bins
from utils import encode_survival, mtlr_neg_log_likelihood, make_optimizer
from utils import mtlr_survival, mtlr_risk, roc_auc_at_times, brier_score_at_times
from prognosis_model import model_lora_again
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR


from ct_clip import CTCLIP
from transformer_maskgit import CTViT
from transformers import BertTokenizer, BertModel
from lifelines.utils import concordance_index
from data_inference_hector import Hector_Dataset_lora

from pycox.models import CoxPH, MTLR, DeepHitSingle
from pycox import models

from peft import get_peft_config, get_peft_model, LoraConfig, TaskType


import sys
sys.argv = ['']

seed = 42
torch.manual_seed(seed) 
generator = torch.Generator().manual_seed(seed)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

tokenizer = BertTokenizer.from_pretrained('microsoft/BiomedVLP-CXR-BERT-specialized',do_lower_case=True)
text_encoder = BertModel.from_pretrained("microsoft/BiomedVLP-CXR-BERT-specialized")

text_encoder.resize_token_embeddings(len(tokenizer))
text_encoder.to(device)

image_encoder = CTViT(
    dim = 512,
    codebook_size = 8192,
    image_size = 480,
    patch_size = 20,
    temporal_patch_size = 10,
    spatial_depth = 4,
    temporal_depth = 4,
    dim_head = 32,
    heads = 8
)

image_encoder.to(device)

clip = CTCLIP(
    image_encoder = image_encoder,
    text_encoder = text_encoder,
    dim_image = 294912,
    dim_text = 768,
    dim_latent = 512,
    extra_latent_projection = False,         # whether to use separate projections for text-to-image vs image-to-text comparisons (CLOOB)
    use_mlm=False,
    downsample_image_embeds = False,
    use_all_token_embeds = False,
)

clip.load("/share/sda/mohammadqazi/project/CTscan_prognosis_VLM-main/docs/CT-CLIP_v2.pt")
clip.to(device)

/home/mohammadqazi/.conda/envs/ct_rate/lib/python3.10/site-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:261: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/home/mohammadqazi/.conda/envs/ct_rate/lib/python3.10/site-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:391: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/home/mohammadqazi/.conda/envs/ct_rate/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/mohammadqazi/.conda/envs/ct_rate/lib/python3.10/site-packages/transformers/modeling_utils.py:463: FutureWarning: You are using `tor

CTCLIP(
  (text_transformer): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.25, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.25, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw

In [6]:
target = [
    "to_q", "to_kv", "to_out"
    # "query", "key", "value"
    ]
peft_config = LoraConfig(
    inference_mode=False, r=2, lora_alpha=32, lora_dropout=0.2, target_modules=target
)

parser = argparse.ArgumentParser(description='Training Script')
parser.add_argument('--name', type=str, default='dummy', help='Name of the experiment')
parser.add_argument('--method', type=str, default='mtlr', help='whether to use mtlr or deephit')
parser.add_argument('--num_time_bins', type=int, default=12, help='NUmber of time bins')

args = parser.parse_args()

model = model_lora_again(clip, device, peft_config, 12)
model.to(device)


hect_dataset = Hector_Dataset_lora(data_folder = "/share/sda/mohammadqazi/project/hector/pre_processed/",  
            csv_file ="/share/sda/mohammadqazi/project/CTscan_prognosis_VLM-main/docs/TNM_hector_prompts.csv", args=args)

train_dataset, test_dataset = hect_dataset.train_val_split(fold=0)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

/home/mohammadqazi/.conda/envs/ct_rate/lib/python3.10/site-packages/pycox/preprocessing/discretization.py:37: UserWarning: cuts are not unique, continue with 10 cuts instead of 12
  warnings.warn(f"cuts are not unique, continue with {len(cuts)} cuts instead of {num}")


In [7]:
img_emb, text_emb, relapse, RFS, _, _, _ = next(iter(train_loader))
img_emb = img_emb.to(device)
text_tokens=tokenizer(text_emb, return_tensors="pt", padding="max_length", truncation=True, max_length=512).to(device)
relapse = relapse.to(device)
RFS = RFS.to(device)
summary(model, input_data=[img_emb, text_tokens ], depth=12, col_names=["input_size", "output_size", "num_params", 'trainable'],)

Layer (type:depth-idx)                                       Input Shape               Output Shape              Param #                   Trainable
model_lora_again                                             [4, 1, 240, 480, 480]     [4, 12]                   --                        Partial
├─CTCLIP: 1-1                                                --                        [4, 24, 24, 512]          302,776,321               Partial
│    └─BertModel: 2-1                                        [4, 512]                  [4, 768]                  --                        False
│    │    └─BertEmbeddings: 3-1                              --                        [4, 512, 768]             --                        False
│    │    │    └─Embedding: 4-1                              [4, 512]                  [4, 512, 768]             (23,440,896)              False
│    │    │    └─Embedding: 4-2                              [4, 512]                  [4, 512, 768]             (1,536)  

In [1]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.shape)

NameError: name 'model' is not defined